# Production Scheduling

In [8]:
import pandas as pd
from pulp import *

In [10]:
df = pd.read_csv('../data/production.csv')
df.columns=['period', 'demand', 'fixed_cost', 'variable_cost', 'storage_cost', 'capacity']
df['time'] = range(1,13)
df.set_index('time', inplace=True)
df.head()

,period,demand,fixed_cost,variable_cost,storage_cost,capacity
time,,,,,,
1,period 1,3000,2000,40,1,4000
2,period 2,4000,2000,40,1,4000
3,period 3,2500,2000,40,1,4000
4,period 4,4000,2000,40,1,4000
5,period 5,6000,2000,40,1,4000


In [11]:
# inventory model variables
inventory = LpVariable.dicts(name='inventory', indices=range(0,13), lowBound=0, upBound=None, cat='Integer')
inventory[0] = 200

# production model variables
production = LpVariable.dicts(name='production', indices=range(1,13), lowBound=0, upBound=None, cat='Integer')

# production status model variables
in_production = LpVariable.dicts(name='inproduction', indices=range(1,13), lowBound=0, upBound=None, cat='Binary')

In [20]:
# set time variable to match df index
time = df.index

In [15]:
model = LpProblem(name='Production', sense=LpMinimize)

In [22]:
# Objective function (minimize)
model += lpSum( [ inventory[t]*df.loc[t,'storage_cost'] + 
                  production[t]*df.loc[t,'variable_cost'] + 
                  in_production[t]*df.loc[t,'fixed_cost'] 
                  for t in time] )

In [23]:
# Constraints
for t in time:
    model += production[t] - inventory[t]  + inventory[t-1] >= df.loc[t, 'demand']
    model += production[t] <= in_production[t] * df.loc[t, 'capacity']


In [65]:
# Solve
solution = model.solve()

# Print the results
if solution == LpStatusOptimal:
    print(f"The minimum cost to meet demand is: ${model.objective.value():,.0f}")

    # find max index
    max_index = df['inventory'].idxmax()

    if max_index == type(list):
        for i in max_index:
            print(f"Maximum inventory will be {df.loc[i, 'inventory']:,.0f} during {df.loc[i, 'period']}") 
    else:
        print(f"Maximum inventory will be {df.loc[max_index, 'inventory']:,.0f} units during {df.loc[max_index, 'period']}") 
    
    print(f"Average inventory will be {df['inventory'].mean():,.0f} units between {df.loc[df.index.min(),'period']} and {df.loc[df.index.max(), 'period']}")

    forecast_inventory=[]
    forecast_production=[]
    production_active=[]
    for i in time: 
        forecast_inventory.append(inventory[i].value())
        forecast_production.append(production[i].value())
        production_active.append(in_production[i].value())          
else:
    print("The model is infeasible or unbounded.")

df['inventory'] = forecast_inventory
df['production'] = forecast_production
df['production_active'] = production_active

df

The minimum cost to meet demand is: $1,373,400
Maximum inventory will be 3,000 units during period 7
Average inventory will be 950 units between period 1 and period 12


,period,demand,fixed_cost,variable_cost,storage_cost,capacity,inventory,production,production_active
time,,,,,,,,,
1,period 1,3000,2000,40,1,4000,600.0,3400.0,1.0
2,period 2,4000,2000,40,1,4000,600.0,4000.0,1.0
3,period 3,2500,2000,40,1,4000,2100.0,4000.0,1.0
4,period 4,4000,2000,40,1,4000,2100.0,4000.0,1.0
5,period 5,6000,2000,40,1,4000,100.0,4000.0,1.0
6,period 6,100,2000,40,1,4000,0.0,0.0,0.0
7,period 7,250,2000,40,1,4000,3000.0,3250.0,1.0
8,period 8,5000,2000,40,1,4000,2000.0,4000.0,1.0
9,period 9,6000,2000,40,1,4000,0.0,4000.0,1.0


In [ ]:
# modifications to this example will be made in production-scheduling-sensitivity.ipynb